In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from smolagents import LiteLLMModel, ToolCallingAgent, CodeAgent


In [ ]:
from opendeepsearch import OpenDeepSearchTool
import os

# accounts/fireworks/models/qwen2p5-72b-instruct
# accounts/fireworks/models/qwq-32b
# accounts/fireworks/models/deepseek-r1-distill-qwen-32b
search_agent = OpenDeepSearchTool(
    model_name="fireworks_ai/accounts/fireworks/models/qwen2p5-72b-instruct",
    reranker='local_jina'
)
if not search_agent.is_initialized:
    search_agent.setup()
    
query = "Fastest land animal?"
result = search_agent.forward(query)
print(result)

In [3]:
from opendeepsearch import OpenDeepSearchTool
# from opendeepsearch.wolfram_tool import WolframAlphaTool
from opendeepsearch.prompts import REACT_PROMPT, MAJORITY_VOTE_PROMPT
from smolagents import LiteLLMModel, ToolCallingAgent, Tool
import os
from opendeepsearch.sc_agent import SelfConsistentAgent
model = LiteLLMModel(
    "fireworks_ai/accounts/fireworks/models/qwen2p5-72b-instruct",  # Your Fireworks Deepseek model
    temperature=0.7
)
search_agent = OpenDeepSearchTool(model_name="fireworks_ai/accounts/fireworks/models/qwen2p5-72b-instruct", reranker="local_jina") # Set reranker to "jina" or "infinity"

# # Initialize the Wolfram Alpha tool
# wolfram_tool = WolframAlphaTool(app_id=os.environ["WOLFRAM_ALPHA_APP_ID"])

# Initialize the React Agent with search and wolfram tools
react_agent = ToolCallingAgent(
    tools=[search_agent],
    model=model,
    prompt_templates=REACT_PROMPT # Using REACT_PROMPT as system prompt
)

judge_agent = ToolCallingAgent(
    tools=[],
    model=model,
    prompt_templates=MAJORITY_VOTE_PROMPT
)

sc_agent = SelfConsistentAgent(
    tool_agent=react_agent,
    judge_agent=judge_agent,
)

# # Example query for the React Agent
# query = "What is the distance, in metres, between the Colosseum in Rome and the Rialto bridge in Venice"
# result = react_agent.run(query)

# print(result)

2025-04-05 20:27:20.998654: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-05 20:27:22.136729: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/rd211/anaconda3/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:984: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/rd211/anaconda3/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1043: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [10]:
import time
# threading
from concurrent.futures import ThreadPoolExecutor
def run_query(query):
    # result = sc_agent.ask_sync(query['Prompt'], n_samples=4)
    
    
    query['our_answer'] = ''
    return query
# with ThreadPoolExecutor(max_workers=4) as executor:
#     queries = [
#         "What is the distance, in metres, between the Colosseum in Rome and the Rialto bridge in Venice",
#         "What is the capital of France?",
#         "How many continents are there on Earth?",
#         "What is the largest mammal?"
#     ]
#     results = list(executor.map(run_query, queries))
# sc_agent.ask_sync("What is the distance, in metres, between the Colosseum in Rome and the Rialto bridge in Venice", n_samples=4)

In [11]:
from datasets import load_dataset

ds = load_dataset('google/frames-benchmark', split='test')

In [12]:
ds = ds.train_test_split(test_size=0.2)

In [13]:
ds = ds['test']

In [15]:
# ds = ds.map(lambda x: {"our_answer": sc_agent.ask_sync(x['Prompt'])}, num_proc=4)
from datasets import Dataset
import pandas as pd
with ThreadPoolExecutor(max_workers=4) as executor:
    # Process each element using threads
    processed_results = list(executor.map(run_query, ds))

# Convert the list of processed results back into a Dataset.
# This assumes that each item in processed_results is a dictionary representing one record.
ds = Dataset.from_pandas(pd.DataFrame(processed_results))


In [9]:
processed_results

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:
from evals.autograde_df import autograde_df


In [ ]:
import pandas as pd

df = pd.read_csv('results.csv')

In [ ]:
df = df[['Prompt', 'Answer', 'our_answer']]

# Rename to original_question, answer, true_answer
df = df.rename(columns={
    'Prompt': 'original_question',
    'Answer': 'true_answer',
    'our_answer': 'answer'
})

df.to_json('eval.json', orient='records', lines=True)

In [3]:
import pandas as pd
import os

files = os.listdir('assets')
# Remove non-json files
files = [f for f in files if f.endswith('.json')]


In [ ]:
# load json files
loaded_dfs = []
for file in files:
    df = pd.read_json(f'assets/{file}', lines=True)
    loaded_dfs.append(df)

['400_500_qwq_seed_43.json',
 '0_100_qwq_seed_43.json',
 '800_824_qwq_seed_43.json',
 '500_600_qwq_seed_43.json',
 '300_400_qwq_seed_43.json',
 '750_800_qwq_seed_43.json',
 '600_700_qwq_seed_43.json',
 '700_750_qwq_seed_43.json',
 '100_200_qwq_seed_43.json',
 '200_300_qwq_seed_43.json']

In [ ]:
autograde_df('eval.json')

In [ ]:
graded_df = pd.read_json('eval.json', lines=True)
# Compute percentage of A, B, C in grade
def compute_percentage(graded_df, grade):
    return (graded_df['final_grade'] == grade).sum() / len(graded_df) * 100
A_percentage = compute_percentage(graded_df, 'A\n')
B_percentage = compute_percentage(graded_df, 'B\n')
C_percentage = compute_percentage(graded_df, 'C\n')
print(f"A: {A_percentage}%")
print(f"B: {B_percentage}%")
print(f"C: {C_percentage}%")

In [ ]:
graded_df['answer'].iloc[3]

In [ ]:
graded_df['true_answer'].iloc[3]

In [ ]:
graded_df['final_grade'].iloc[3]

In [ ]:
import fasttext
from huggingface_hub import hf_hub_download
import wikipediaapi

# Load the model
model = fasttext.load_model(hf_hub_download("kenhktsui/llm-data-textbook-quality-fasttext-classifer-v2", "model.bin"))
